<a href="https://colab.research.google.com/github/ShivM99/Python-ML/blob/main/Titanic_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the training dataset
import pandas as pd
train = pd.read_csv (r"titanic_train.csv")
x = train.iloc[:, [2, 4, 5, 6, 7, 9, 11]].values
y = train.iloc[:, 1].values
print ("Features:\n", x)
print ("\nPredictor:\n", y)

Features:
 [[3 'male' 22.0 ... 0 7.25 'S']
 [1 'female' 38.0 ... 0 71.2833 'C']
 [3 'female' 26.0 ... 0 7.925 'S']
 ...
 [3 'female' nan ... 2 23.45 'S']
 [1 'male' 26.0 ... 0 30.0 'C']
 [3 'male' 32.0 ... 0 7.75 'Q']]

Predictor:
 [0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 0

In [ ]:
#Handling the missing values
#One hot encoding the categorical features
print ("\nMissing values in different features:\n", train.isnull().sum())
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#ColumnTransformer (transformers = [("name_of_object", object_creation, [columns]), ("name_of_object", object_creation, [columns]), ...], remainder = "passthrough")
ct = ColumnTransformer (transformers = [("si_mean", SimpleImputer (missing_values = np.nan, strategy = "mean"), [2]), ("encoder", OneHotEncoder(), [0, 1])], remainder = "passthrough")
x = ct.fit_transform (x)
#Pipeline (steps = [("name_of_object", object_creation), ("name_of_object", object_creation), ...])
pipe = Pipeline (steps = [("si_mode", SimpleImputer (missing_values = np.nan, strategy = "most_frequent")), ("encoder", OneHotEncoder())])
embarked = ColumnTransformer (transformers = [("pipe", pipe, [-1])], remainder = "passthrough")
x = embarked.fit_transform (x)
x_df = pd.DataFrame (x) #Converting 'x' (array) into data frame to see every column in the order they are occuring in 'x'
print ("\nPreprocessed features:\n", x_df)


Missing values in different features:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Preprocessed features:
       0    1    2          3    4    5    6    7    8  9  10       11
0    0.0  0.0  1.0       22.0  0.0  0.0  1.0  0.0  1.0  1  0     7.25
1    1.0  0.0  0.0       38.0  1.0  0.0  0.0  1.0  0.0  1  0  71.2833
2    0.0  0.0  1.0       26.0  0.0  0.0  1.0  1.0  0.0  0  0    7.925
3    0.0  0.0  1.0       35.0  1.0  0.0  0.0  1.0  0.0  1  0     53.1
4    0.0  0.0  1.0       35.0  0.0  0.0  1.0  0.0  1.0  0  0     8.05
..   ...  ...  ...        ...  ...  ...  ...  ...  ... .. ..      ...
886  0.0  0.0  1.0       27.0  0.0  1.0  0.0  0.0  1.0  0  0     13.0
887  0.0  0.0  1.0       19.0  1.0  0.0  0.0  1.0  0.0  0  0     30.0
888  0.0  0.0  1.0  29.699118  0.0  0.0  1.0  1.0  0.0  1  2    

In [ ]:
#Splitting the dataset into training & test datasets not done as the dataset is already splitted
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size = 0.2, random_state = 0)

In [ ]:
#Building the XGBC model
from sklearn.ensemble import GradientBoostingClassifier as xGBC
xgb = xGBC ()
xgb.fit (x_train, y_train)
y_pred = xgb.predict (x_test)

In [ ]:
#Checking the accuracy of XGBC model
from sklearn.metrics import accuracy_score
print ("\nAccuracy of the xGBC model:", accuracy_score (y_test, y_pred)*100)


Accuracy of the xGBC model: 83.79888268156425


In [ ]:
#Cross-validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score (estimator = xgb, X = x_train, y = y_train, cv = 10)
print ("\nMean accuracy:", accuracies.mean()*100)


Mean accuracy: 82.16353677621284


In [ ]:
#Grid search
from sklearn.model_selection import GridSearchCV
h_parameters = [{"learning_rate": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], "n_estimators": [100, 200, 300, 400, 500], "subsample": [0.25, 0.5, 0.75, 1.0],  "max_depth": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20], "min_samples_split": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20], "min_samples_leaf": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20], "max_features": ["auto", "sqrt", "log2", "None"], "max_leaf_nodes": [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}]
grid = GridSearchCV (estimator = xgb, param_grid = h_parameters, scoring = "accuracy", n_jobs = -1, cv = 10)
grid.fit (x_train, y_train)
best_hparameters = grid.best_params_
best_accuracy = grid.best_score_
print ("\nBest hyper-parameters:\n", best_hparameters)
print ("\nBest accuracy:", best_accuracy*100)